In [1]:
import numpy as np
import pandas as pd
from pathmgmt import pathmgmt as myPath
import os

In [2]:
# Combine all alphas
def get_stat():
    res = pd.DataFrame()
    for folderName in os.listdir(myPath.PLOT_DIR):
        if 'zz9999' not in folderName or 'Lqd' not in folderName:
            continue
        # print(folderName)
        folder = myPath.PLOT_DIR/folderName/'statistics'
        for fileName in os.listdir(folder):
            # print(fileName)
            file = folder/fileName
            df = pd.read_csv(file)
            df.index = [f'{folderName};{fileName[:-15]}']
            # print(df)
            if res.empty:
                res = df
            else:
                res = pd.concat([res, df])
    return res

In [3]:
stats = get_stat()

In [4]:
stats.describe()

,Annualized Return,Annualized Excess Return,IR,IR long only,IC,Max Drawdown,Max Drawdown long only,daily Turnover
count,562.000000,562.000000,562.000000,562.000000,562.000000,562.000000,562.000000,562.000000
mean,0.145569,0.008578,2.706276,0.069510,0.012195,0.069305,0.171880,0.231212
std,0.150991,0.027210,1.669375,0.303862,0.008595,0.066746,0.060973,0.325789
min,-0.086086,-0.085206,-0.848314,-0.931348,-0.003659,0.001911,0.074962,0.008278
25%,0.045773,-0.009246,1.324191,-0.104194,0.005420,0.022031,0.127691,0.031577
50%,0.101186,0.013834,2.990833,0.130986,0.012212,0.052657,0.154376,0.089640
75%,0.200956,0.027107,3.883308,0.214906,0.017637,0.086470,0.193618,0.270888
max,0.867974,0.072142,6.474684,0.982315,0.037569,0.358252,0.335895,1.377221


In [5]:
good_alphas = list(stats.loc[(stats.IR > 3.5) & (stats['daily Turnover'] < 0.2)].index)

In [6]:
len(good_alphas)

70

In [7]:
# combine daily pnl
def get_pnl():
    res = pd.DataFrame()
    for folderName in os.listdir(myPath.PLOT_DIR):
        if 'zz9999' not in folderName and 'Lqd' not in folderName:
            continue
        # print(folderName)
        folder = myPath.PLOT_DIR/folderName/'PnL_results'
        for fileName in os.listdir(folder):
            file = folder/fileName
            df = pd.read_csv(file)
            # df = df.set_index('time')
            # df = df['pnl']
            df = df[['time', 'pnl']]
            df.rename(
                columns={'pnl': f'{folderName};{fileName[:-13]}'}, inplace=True)
            if res.empty:
                res = df
            else:
                res = res.merge(df, on='time', how='outer')
    res = res.set_index('time').sort_index()
    return res


In [8]:
pnls = get_pnl()

In [9]:
corr = pnls[good_alphas].corr()

In [32]:
from itertools import combinations
import random
res = []
num = 6
good_candidates = list(
    ((corr.mask(np.eye(len(corr), dtype=bool)).abs() < 0.6).sum() > num).index)
random.shuffle(good_candidates)

In [33]:
len(good_candidates)

70

In [ ]:
for cols in combinations(good_candidates, num):
    corr_small = corr.loc[cols, cols]
    if (~(corr_small.mask(np.eye(len(corr_small), dtype=bool)).abs() > 0.6).any()).sum() == num:
        res.append(corr_small)
        break

In [27]:
res[0]

,alpha.YaoLqd004-10days-zz9999;20180101-20201231-None-Industry-holding10days,alpha.YaoLqd008-20days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoLqd003-20days-zz9999;20180101-20201231-None-Industry-holding60days,alpha.YaoLqd010-20days-zz9999;20180101-20201231-None-Industry-holding10days,alpha.YaoLqd009-10days-zz9999;20180101-20201231-None-Industry-holding5days,alpha.YaoLqd010-20days-zz9999;20180101-20201231-None-Industry-holding60days
alpha.YaoLqd004-10days-zz9999;20180101-20201231-None-Industry-holding10days,1.000000,0.258408,0.597251,-0.117040,0.218823,-0.094056
alpha.YaoLqd008-20days-zz9999;20180101-20201231-None-Industry-holding1days,0.258408,1.000000,0.576838,-0.139407,-0.043294,-0.231230
alpha.YaoLqd003-20days-zz9999;20180101-20201231-None-Industry-holding60days,0.597251,0.576838,1.000000,-0.146538,0.014820,-0.337449
alpha.YaoLqd010-20days-zz9999;20180101-20201231-None-Industry-holding10days,-0.117040,-0.139407,-0.146538,1.000000,0.413072,0.581626
alpha.YaoLqd009-10days-zz9999;20180101-20201231-None-Industry-holding5days,0.218823,-0.043294,0.014820,0.413072,1.000000,0.585661
alpha.YaoLqd010-20days-zz9999;20180101-20201231-None-Industry-holding60days,-0.094056,-0.231230,-0.337449,0.581626,0.585661,1.000000


In [28]:
stats.loc[stats.index.isin(res[0].index)]

,Annualized Return,Annualized Excess Return,IR,IR long only,IC,Max Drawdown,Max Drawdown long only,daily Turnover
alpha.YaoLqd003-20days-zz9999;20180101-20201231-None-Industry-holding60days,0.055791,0.037103,3.611858,0.219739,0.017851,0.012688,0.269240,0.023003
alpha.YaoLqd004-10days-zz9999;20180101-20201231-None-Industry-holding10days,0.123419,0.021545,3.524776,0.190650,0.018196,0.034516,0.177852,0.143407
alpha.YaoLqd008-20days-zz9999;20180101-20201231-None-Industry-holding1days,0.523000,0.058752,3.729724,0.695988,0.020512,0.084241,0.129483,0.051708
alpha.YaoLqd009-10days-zz9999;20180101-20201231-None-Industry-holding5days,0.347363,-0.024068,3.959844,-0.276286,0.020893,0.054443,0.159512,0.097277
alpha.YaoLqd010-20days-zz9999;20180101-20201231-None-Industry-holding10days,0.105815,0.032416,5.311262,0.278044,0.015812,0.011912,0.200846,0.142707
alpha.YaoLqd010-20days-zz9999;20180101-20201231-None-Industry-holding60days,0.024551,0.037136,3.976771,0.202426,0.012889,0.006757,0.297668,0.023147


In [35]:
# max_IR = float('-inf')
# max_idx = -1
# for i in range(len(res)):
#     total_IR = np.sum([(stats.loc[stats.index==name].IR) for name in res[i].index])
#     if total_IR > max_IR:
#         max_IR = total_IR
#         max_idx = i

In [36]:
# res[max_idx]

In [37]:
# stats.loc[stats.index.isin(res[max_idx].index)]